In [1]:
import pandas as pd
import numpy as np
import torch
import copy
from tqdm import tqdm
import datetime

In [2]:
scenario_requirements = {
    1:[1320000, 1400000],
    2:[1200000, 1300000],
    3:[1370000, 1260000],
}

In [3]:
SCENARIO_NUMBER = 1
min_quantile_10 = scenario_requirements[SCENARIO_NUMBER][0]
min_quantile_25 = scenario_requirements[SCENARIO_NUMBER][1]
best_strat = []
best_profit = 0
options = {
    0:(5.0,'put'),
    1:(5.5,'put'),
    2:(6.0,'put'),
    3:(6.5,'put'),
    4:(7.0,'put'),
    5:(7.5,'put'),
    6:(8.0,'put'),
    7:(5.0,'call'),
    8:(5.5,'call'),
    9:(6.0,'call'),
    10:(6.5,'call'),
    11:(7.0,'call'),
    12:(7.5,'call'),
    13:(8.0,'call'),
}

In [4]:
scenario = pd.read_csv('104/Scenario'+ str(SCENARIO_NUMBER) +'.csv')
prices = pd.read_csv('104/option_prices_scenario_'+ str(SCENARIO_NUMBER) + '.csv')
prices_list = prices['Price [USD]'].to_list()
scenario_list = scenario.to_dict(orient='list')

In [5]:
def get_vectors_closer(vec1, vec2):
  sum = np.add(vec1, vec2)
  sum = sum * 0.5
  return sum
  
def closer_one_step(vec1, vec2):
  vec = get_vectors_closer(vec1, vec2)
  return get_vectors_closer(vec1, vec)

def closer_sbs(vec1, vec2, n):
  vec = closer_one_step(vec1, vec2)
  for i in range(n-1):
    vec = closer_one_step(vec1, vec)
  return vec

In [6]:
def calculate_profit(strike, market_price, typ):
    if typ == 'call':
        return max(100 * (market_price - strike), 0.0)
    return max(100 * (strike - market_price), 0.0)

In [7]:
def profit(option_prices, yi, market_price, strategy):
    res = yi * market_price * 1000
    before_options = res
    for i in range(14):
        strike, typ = options[i]
        res += strategy[i] * calculate_profit(strike, market_price, typ)
        res -= option_prices[i] * strategy[i]
    return res

### Saska es Huni

In [43]:
number_of_random_strategies = 100000
best_quantile_10 = 0
for i in tqdm(range(number_of_random_strategies)):
#     strat = np.random.randint(100, size=14)
    strat = np.hstack((np.random.randint(2000, size=7), np.zeros(7)))
    if (best_strat != []):
      strat = np.round(closer_sbs(best_strat, strat, 2))
#     strat = [1000, 1000, 1000, 100, 1000, 100, 0, 0, 0, 0, 0, 0, 0, 0]
    curr_profits = []
    for j in range(1000):
        curr_strat_profit = profit(prices_list, scenario_list['yield [thousands of bushels]'][j], scenario_list['price [$/bushel]'][j], strat)
#         print(curr_strat_profit)
#         print(sc1_list['yield [thousands of bushels]'][j])
#         print(sc1_list['price [$/bushel]'][j])
        curr_profits.append(curr_strat_profit)
    curr_expectet_profit = np.mean(curr_profits)
    quantile_10 = np.quantile(curr_profits, 0.1)
    quantile_25 = np.quantile(curr_profits, 0.25)
    if curr_expectet_profit > best_profit and quantile_10 > min_quantile_10 and quantile_25 > min_quantile_25:
        best_profit = curr_expectet_profit
        best_strat = strat
        print(quantile_10)
        print(quantile_25)
        print('New best strategy at iteration ',i,' : ', strat, 'Expected Profit:', curr_expectet_profit)

  0%|                                                                                       | 0/100000 [00:00<?, ?it/s]C:\Users\peter\AppData\Local\Temp\ipykernel_26288\1578977321.py:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if (best_strat != []):
  1%|▍                                                                            | 578/100000 [00:06<17:57, 92.28it/s]

1325019.1199
1400320.9688999997
New best strategy at iteration  561  :  [600. 498. 500. 517. 554. 347. 264.   0.   0.   0.   0.   0.   0.   0.] Expected Profit: 1558547.1996


  4%|██▋                                                                         | 3612/100000 [00:38<16:56, 94.79it/s]

1325127.7754999998
1400266.6839999997
New best strategy at iteration  3595  :  [583. 525. 476. 502. 571. 332. 273.   0.   0.   0.   0.   0.   0.   0.] Expected Profit: 1558603.1084999999


  8%|██████▎                                                                     | 8225/100000 [01:29<16:24, 93.18it/s]

1325215.2566000002
1400018.7896000003
New best strategy at iteration  8207  :  [549. 521. 521. 510. 554. 346. 258.   0.   0.   0.   0.   0.   0.   0.] Expected Profit: 1558642.6753000002


  9%|██████▋                                                                     | 8743/100000 [01:35<16:49, 90.43it/s]

1325621.9612000003
1400147.3742000002
New best strategy at iteration  8729  :  [535. 507. 512. 538. 561. 336. 254.   0.   0.   0.   0.   0.   0.   0.] Expected Profit: 1558693.2204000002


 15%|███████████                                                                | 14699/100000 [02:41<15:36, 91.11it/s]


KeyboardInterrupt: 

### Akos

In [8]:
def get_expected_value_of_strategy(strategy, market_prices, yields, option_prices):
    profits = []
    for i in range(1000):
        curr_profit = profit(option_prices, yields[i], market_prices[i], strategy)
        profits.append(curr_profit)
    expected_profit = np.mean(profits)
    quantile_10 = np.quantile(profits, 0.1)
    quantile_25 = np.quantile(profits, 0.25)
    if quantile_10 > min_quantile_10 and quantile_25 > min_quantile_25:
        return expected_profit
    return 0

In [9]:
def get_similar_strategies(strat):
    ret = []
    for i in range(14):
        temp1 = copy.deepcopy(strat)
        temp2 = copy.deepcopy(strat)
        temp1[i] = strat[i] +1
        temp2[i] -= 1
        if np.min(temp1) >= 0:
            ret.append(temp1)
        if np.min(temp2) >= 0:
            ret.append(temp2)
    return ret

In [ ]:
filename = "results_scenario" + str(SCENARIO_NUMBER) + ".txt"
number_of_random_strategies = 1000000
# best_profit = 0
best_quantile_10 = 0
curr_strat = np.hstack((np.random.randint(2000, size=7), np.zeros(7)))
best_expected_profit = 0
prev_best = -1
for i in tqdm(range(number_of_random_strategies)):
    prev_best = best_expected_profit
    curr_profits = []
    similar_strategies = get_similar_strategies(curr_strat)
    for strat in similar_strategies:
        curr_strat_expected_profit = get_expected_value_of_strategy(strat, scenario_list['price [$/bushel]'], scenario_list['yield [thousands of bushels]'], prices_list)
        if curr_strat_expected_profit > best_expected_profit:
            best_expected_profit = curr_strat_expected_profit
            curr_strat = strat
#             print('New best strategy at iteration ',i,' : ', strat, 'Expected Profit:', curr_strat_expected_profit)
    if prev_best == best_expected_profit:
        if best_expected_profit != 0:
            print('Start:', curr_strat, "Expected Profit: ", best_expected_profit)
            to_write = ''
            for e in curr_strat:
                to_write += str(e) + ' '
            to_write = to_write + ',' + str(best_expected_profit) + '\n'
            f = open(filename, "a")
            f.write(to_write)
            f.close()
        curr_strat = np.hstack((np.random.randint(2000, size=7), np.zeros(7)))
        best_expected_profit = 0


  0%|                                                                         | 471/1000000 [01:42<55:38:36,  4.99it/s]

Start: [242. 954. 495.  36. 949. 619.   0.   0.   0.   0.   0.   0.   0.   0.] Expected Profit:  1558217.6302


  0%|                                                                        | 1247/1000000 [04:28<58:23:18,  4.75it/s]

Start: [553. 483.  98. 873. 126. 412. 457.   0.   0.   0.   0.   0.   0.   0.] Expected Profit:  1558323.0052999996


  0%|▏                                                                       | 1942/1000000 [06:59<59:26:31,  4.66it/s]

...

  0%|▎                                                                       | 3955/1000000 [14:12<64:58:30,  4.26it/s]

Start: [ 159.  558.  186. 1204.  431.  129.  257.    0.    0.    0.    0.    0.
    0.    0.] Expected Profit:  1560168.32


  0%|▎                                                                       | 3991/1000000 [14:20<55:47:56,  4.96it/s]

Start: [ 856.   54.   36. 1158.  676.  463.    0.    0.    0.    0.    0.    0.
    0.    0.] Expected Profit:  1558877.8365


  0%|▎                                                                       | 4500/1000000 [16:10<58:20:22,  4.74it/s]